In [1]:
import os
import subprocess
import pandas as pd

# Пути Windows заменены на пути WSL
ligand_path = '/home/ardea/docking_3/11465/alpha_6b.pdbqt'        # один лиганд
receptor_path = '/mnt/c/Users/User/docking_7/01734_rigid.pdbqt'
flex_path = '/mnt/c/Users/User/docking_7/01734_flex.pdbqt'          # файл гибких остатков
gtv = '/home/ardea/VC_1_0/autodock_vina_1_1_2/build/linux/release/vina-carb'
csv_path = '/home/ardea/docking_7/01734/6/poses_energies_carb_flex.csv'
results = []

# Параметры поиска
# Параметры поиска (укажите свои)
center_x, center_y, center_z = 11.988, -1.854, 5.298
size_x, size_y, size_z = 44, 68, 88
exhaustiveness = 1
num_modes = 10
num_runs = 10

for run_id in range(1, num_runs + 1):
    config_path = f'/mnt/c/Users/User/docking_7/01734/6/config_run_{run_id}.conf'
    out_path = f'/mnt/c/Users/User/docking_7/01734/6/out_run_{run_id}.pdbqt'
    log_path = f'/mnt/c/Users/User/docking_7/01734/6/log_run_{run_id}.txt'

    with open(config_path, 'w') as conf_file:
        conf_file.write(f"""receptor = {receptor_path}
flex = {flex_path}
ligand = {ligand_path}
center_x = {center_x}
center_y = {center_y}
center_z = {center_z}
size_x = {size_x}
size_y = {size_y}
size_z = {size_z}
out = {out_path}
exhaustiveness = {exhaustiveness}
score_only = 0
chi_coeff = 0.0
chi_cutoff = 12.0
num_modes = {num_modes}
seed = {12345 + run_id}
""")

    cmd = [gtv, '--config', config_path]
    print(f"Запуск докинга #{run_id}...")
    try:
        with open(log_path, 'w') as log_file:
            subprocess.run(cmd, stdout=log_file, stderr=log_file, check=True)
        energies = []
        with open(log_path, 'r') as log_file:
            start_parsing = False
            for line in log_file:
                line = line.strip()
                if line.startswith('-----+'):
                    start_parsing = True
                    continue
                if start_parsing:
                    if line and line[0].isdigit():
                        parts = line.split()
                        if len(parts) >= 2:
                            mode_num = int(parts[0])
                            affinity = float(parts[1])
                            energies.append((mode_num, affinity))
                    else:
                        break
        if not energies:
            print(f"Не удалось найти энергии в {log_path}")
            energies = [(None, float('nan'))]
        for mode_num, affinity in energies:
            results.append({'run': run_id,
                            'pose': os.path.basename(ligand_path),
                            'mode': mode_num,
                            'energy': affinity})
        print(f"Run #{run_id} energies: {[e[1] for e in energies]}")
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при докинге запуска #{run_id}: {e}")

if results:
    pd.DataFrame(results).to_csv(csv_path, index=False)
    print(f"Все результаты сохранены в {csv_path}")
else:
    print("Результаты не получены.")

Запуск докинга #1...
Run #1 energies: [-17.9, -17.8, -17.1, -16.7, -16.7, -16.6, -16.6, -16.3, -16.2, -15.6]
Запуск докинга #2...
Run #2 energies: [-17.9, -17.6, -16.7, -16.6, -16.5, -16.0, -16.0, -15.4, -15.4, -15.3]
Запуск докинга #3...
Run #3 energies: [-17.9, -17.9, -16.9, -16.7]
Запуск докинга #4...
Run #4 energies: [-17.8, -17.8, -17.4, -16.9, -16.7, -16.1, -16.0, -16.0, -15.9, -15.8]
Запуск докинга #5...
Run #5 energies: [-17.5, -16.6, -16.4, -14.5]
Запуск докинга #6...
Run #6 energies: [-17.8, -17.7, -17.4, -16.8, -16.7, -16.4, -16.2, -16.1, -16.1, -16.1]
Запуск докинга #7...
Run #7 energies: [-17.9, -17.8, -16.8, -16.8, -16.7, -16.6, -16.3, -16.1, -15.9, -15.8]
Запуск докинга #8...
Run #8 energies: [-17.8, -17.1, -16.3, -16.2, -15.5, -15.5, -15.4, -15.3, -15.3, -15.2]
Запуск докинга #9...
Run #9 energies: [-17.9, -17.8, -17.1, -16.2, -16.0, -15.8, -15.8, -15.8, -15.7, -15.6]
Запуск докинга #10...
Run #10 energies: [-17.9, -17.1, -16.7, -16.7, -16.6, -16.6, -16.3, -16.2, -16.0,

OSError: Cannot save file into a non-existent directory: '/home/ardea/docking_7/01734/6'

In [ ]:
'/mnt/c/Users/User/docking_7/02229/8/poses_energies_carb_rigid.csv'

In [2]:
new_csv_path = '/mnt/c/Users/User/docking_7/01734/6/poses_energies_carb_rigid.csv'
pd.DataFrame(results).to_csv(new_csv_path, index=False)
print(f"Все результаты сохранены в {new_csv_path}")


Все результаты сохранены в /mnt/c/Users/User/docking_7/01734/6/poses_energies_carb_rigid.csv
